In [1]:
!pip freeze | grep scikit-learn

'grep' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import pickle
import pandas as pd

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

c:\Users\andre\anaconda3\envs\mlflow\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\andre\anaconda3\envs\mlflow\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [5]:
year = 2021
month = 2
df = read_data(f'https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_{year:04d}-{month:02d}.parquet')

In [6]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [7]:
y_pred.mean()

16.191691679979066

In [8]:
df_result = pd.DataFrame()
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
df_result['ride_id'] = df['ride_id']
df_result['actual_duration'] = df['duration']
df_result['predicted_duration'] = y_pred
df_result['diff'] = abs(df_result['predicted_duration'] - df_result['actual_duration'])

In [9]:
df_result.head()

,ride_id,actual_duration,predicted_duration,diff
1,2021/02_1,10.666667,14.539865,3.873198
2,2021/02_2,14.566667,13.740422,0.826244
3,2021/02_3,7.950000,15.593339,7.643339
4,2021/02_4,13.800000,15.188118,1.388118
5,2021/02_5,8.966667,13.817206,4.850539


In [10]:
df_result.to_parquet(
    './results.parquet',
    engine='pyarrow',
    compression=None,
    index=False
)